# 📊 Tehran Rent Market Analysis (1403) – Data Cleaning

## 🎯 Objective
The goal of this notebook is to clean and normalize the raw rental listings data collected from Divar, covering more than **50,000 records** of rental and mortgage advertisements in Tehran (2024/2025).

## 📂 Input
- Raw CSV file of rental listings  
  (columns include: neighborhood_slug, size (sqm), rent_value, credit_value, created_month, etc.)

## 📄 Output
- Cleaned and normalized DataFrame ready for analysis  
- Saved as a new CSV file: `tehran_rent_cleaned.csv`

## 🛠 Tools & Libraries
- Python
- Pandas
- Numpy
- Regular Expressions

✅ In this step we remove missing or invalid records, change date format (from gregorian to jalali) and ensure numeric consistency in key columns such as size and prices.


In [506]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import re
import jdatetime

## prepare file
- read original file and get rental dataframe to work

In [507]:
source_file=pd.read_csv(r"C:\Users\arezoo\Desktop\divar_last_edit\rent_data\real_estate_ads_in_tehran.csv",encoding='utf-8',low_memory=False)

In [508]:
source_file = source_file.drop_duplicates()

In [510]:
rent_df=source_file[source_file["cat2_slug"]=="residential-rent"].copy()

## create date column
- change gregorian_date to jalali_date
- extract year- month and day

In [512]:
rent_df["created_at_month"]=pd.to_datetime(rent_df["created_at_month"].astype(str))

In [513]:
def to_jalali(gregorian_date):
    if pd.isnull(gregorian_date):
        return None
    return jdatetime.date.fromgregorian(date=gregorian_date).strftime('%Y-%m-%d')

In [514]:
rent_df["jalali_date"]=rent_df["created_at_month"].apply(to_jalali)

In [515]:
rent_df[['jalali_year', 'jalali_month', 'jalali_day']] = rent_df['jalali_date'].str.split('-', expand=True)

In [516]:
rent_df["jalali_year"]=pd.to_numeric(rent_df["jalali_year"],errors='coerce')

In [518]:
rent_df["jalali_month"]=rent_df["jalali_month"].apply(lambda x:x[1] if x[0]=="0" else x)

## Convert Persian Digits and Letters to English
- normalize all numeric columns and textual identifiers to use English characters instead of Persian (Arabic) ones

In [519]:
def clean_row(value):
    try:
        value = str(value).strip().replace('‌', '').replace(' ', '')
        translation_table = str.maketrans("۰۱۲۳۴۵۶۷۸۹", "0123456789")
        value = value.translate(translation_table)
        return float(value)
    except Exception as e:
        return np.nan

In [520]:
rent_df["construction_year"]=rent_df["construction_year"].apply(lambda x:"1360"if x=="قبل از ۱۳۷۰"else x)

In [521]:
rent_df["construction_year"]=rent_df["construction_year"].apply(clean_row)

In [522]:
rent_df['rooms_count'] = rent_df['rooms_count'].astype(str).str.strip().replace(r'\s+', '', regex=True)

In [523]:
room_map = {
    'بدوناتاق': 0,
    'یک': 1,
    'دو': 2,
    'سه': 3,
    'چهار': 4,
    'پنج یا بیشتر': 5,
}
rent_df['rooms_count_num'] = rent_df['rooms_count'].astype(str).str.strip().map(room_map)

In [524]:
cols=["rooms_count_num","construction_year","jalali_month","jalali_year","jalali_day","building_size","rent_value","credit_value"]
for col in cols:
    rent_df.loc[:,col]=pd.to_numeric(rent_df.loc[:,col],errors='coerce')

## Add Derived Fields
- create new derived fields to furture calculates.
Adding this feature enables meaningful comparisons between listings and provides a single metric for clustering and analysis

In [525]:
def is_fake_price(n):
    try:
        if pd.isna(n): return False
        s = str(int(n))
    except:
        return True

    # الگوهای تکراری مثل 111111111
    if re.fullmatch(r'(\d)\1{6,}', s):  # حداقل 6 بار تکرار یک رقم
        return True

    # عددهای خیلی بزرگ غیرواقعی
    if len(s) >= 12:
        return True

    return False

In [526]:
rent_df.loc[:,"fake_rent"]=rent_df.loc[:,"rent_value"].apply(is_fake_price)
rent_df.loc[:,"fake_credit"]=rent_df.loc[:,"credit_value"].apply(is_fake_price)

In [527]:
rent_df.loc[:,"total_cost"]=rent_df.loc[:,"credit_value"] + rent_df.loc[:,"rent_value"]/12

In [528]:
rent_df.loc[:,"building_age"]=rent_df.loc[:,"jalali_year"]-rent_df.loc[:,"construction_year"]
rent_df.loc[:,'has_warehouse'] = rent_df.loc[:,'has_warehouse'].apply(
    lambda x: 1 if str(x).strip().lower() == 'true' else 0
)
rent_df.loc[:,'has_parking'] = rent_df.loc[:,'has_parking'].apply(
    lambda x: 1 if str(x).strip().lower() == 'true' else 0
)

In [529]:
conditions = [
    (rent_df["jalali_month"].isin([1,2,3])),(rent_df["jalali_month"].isin ([4,5,6])),
    (rent_df["jalali_month"].isin ([7,8,9])), (rent_df["jalali_month"].isin ([10,11,12]))
]

choices = ["spring","summer","autumn","winter"]
rent_df.loc[:,"season"] = np.select(conditions, choices)

In [530]:
rent_df.loc[:,"cost_per_sqm"]=rent_df.loc[:,"total_cost"]/rent_df.loc[:,"building_size"]

In [531]:
cols=["rooms_count_num","construction_year","jalali_month","jalali_year","jalali_day","total_cost","building_size","rent_value","credit_value"]
for col in cols:
    rent_df.loc[:,col]=pd.to_numeric(rent_df.loc[:,col],errors='coerce')

## Filter Data Below 95th Percentile Threshold
- remove extreme outliers by keeping only records where the value of a numeric column is below its **95th percentile**

In [532]:
def calculate_high_cost_percentage(df, neighborhood_col, cost_col, percentile=95):
    final_result = pd.DataFrame()  # تعریف اولیه

    neighborhoods = df[neighborhood_col].unique()
    
    for n in neighborhoods:
        subset = df[df[neighborhood_col] == n]
        
        if len(subset) == 0:
            continue

        # محاسبه آستانه پرت
        threshold = subset[cost_col].quantile(percentile / 100)
        
        # فقط داده‌هایی که زیر آستانه هستن نگه می‌داریم
        subset_filtered = subset[subset[cost_col] <= threshold]

        # به نتیجه نهایی اضافه می‌کنیم
        final_result = pd.concat([final_result, subset_filtered], ignore_index=True)

    return final_result


In [533]:
rent_df=calculate_high_cost_percentage(rent_df,"neighborhood_slug","total_cost",percentile=95)

## Final Data Filtering: Valid Listings Only
- apply a set of filters to keep only realistic and meaningful rental listings
- enforce reasonable thresholds on key fields and exclude records marked as fake

In [534]:
rent_df = rent_df[
    (rent_df["building_size"] >= 30) &
    (rent_df["building_size"] <= 400) &
    (rent_df["rent_value"] < rent_df["credit_value"] * 0.5) &
    (~rent_df["fake_rent"]) &
    ((~rent_df["fake_credit"]) & (rent_df["rent_value"] >= 2_000_000))
]

In [504]:
ads_count=rent_df["neighborhood_slug"].value_counts()
valid_neighborhoods =ads_count[ads_count>=10].index
rent_df=rent_df[rent_df["neighborhood_slug"].isin(valid_neighborhoods)]

## Export Cleaned Data for Next Steps
- save the cleaned and filtered dataset to a CSV file for reuse in the next stages of analysis

In [110]:
rent_df.to_csv(r"C:\Users\a.zare.PM\Desktop\divar_last_edit\rent_data\data_to_check.csv")